In [1]:
import sys
from pathlib import Path
import polars as pl
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import recall_score, make_scorer

parent_dir = Path.cwd().parent
project_src = f"{parent_dir}/src"

if str(project_src) not in sys.path:
    sys.path.append(str(project_src))

In [2]:
from factory import create_default_data_pipeline
from config import settings

In [3]:
dataset_pipeline = create_default_data_pipeline()

In [4]:
dataset_pipeline.df.describe()

statistic,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,TimeHours,HourOfDay,TimeSincePrevSec,TxnTimeSec,isNight,amountLog,amountBin
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,283726.0,"""283726"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",94811.0776,0.005917,-0.004135,0.001613,-0.002966,0.001828,-0.001139,0.001801,-0.000854,-0.001596,-0.001441,0.000202,-0.000715,0.000603,0.000252,0.001043,0.001162,0.00017,0.001515,-0.000264,0.000187,-0.000371,-0.000015,0.000198,0.000214,-0.000232,0.000149,0.001763,0.000547,88.472687,0.001667,14.537139,26.33641,0.60901,52333.70154,1.0,3.15376,null
"""std""",47481.047891,1.948026,1.646703,1.508682,1.414184,1.377008,1.331931,1.227664,1.179054,1.095492,1.076407,1.01872,0.994674,0.99543,0.952215,0.914894,0.873696,0.842507,0.837378,0.813379,0.769984,0.723909,0.72455,0.623702,0.605627,0.52122,0.482053,0.395744,0.328027,250.399437,0.040796,5.846094,13.18918,1.054718,21045.940171,0.0,1.65708,null
"""min""",0.0,-56.40751,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-43.557242,-73.216718,-13.434066,-24.588262,-4.797473,-18.683715,-5.791881,-19.214325,-4.498945,-14.129855,-25.162799,-9.498746,-7.213527,-54.49772,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"""10-50"""
"""25%""",54204.0,-0.915954,-0.600324,-0.889684,-0.850137,-0.689836,-0.769031,-0.552516,-0.208828,-0.644221,-0.535582,-0.76165,-0.4062,-0.647867,-0.425735,-0.58146,-0.46686,-0.483931,-0.498016,-0.456291,-0.211471,-0.228305,-0.542703,-0.161705,-0.354456,-0.317488,-0.326764,-0.070642,-0.052818,5.6,0.0,10.5975,15.056667,0.0,38151.0,1.0,1.88707,null
"""50%""",84693.0,0.020386,0.063952,0.179967,-0.02224,-0.053466,-0.275167,0.040862,0.021899,-0.052596,-0.093236,-0.032302,0.139074,-0.012917,0.050216,0.0493,0.067119,-0.065865,-0.002136,0.003371,-0.062353,-0.02944,0.006675,-0.011157,0.041016,0.016281,-0.05217,0.001479,0.011288,22.0,0.0,15.008889,23.525833,0.0,54032.0,1.0,3.135494,null
"""75%""",139298.0,1.316069,0.800283,1.026962,0.739654,0.612223,0.396794,0.570475,0.325708,0.595978,0.45362,0.739582,0.616979,0.663182,0.492336,0.65011,0.523512,0.398974,0.501956,0.458509,0.133208,0.186195,0.528245,0.147749,0.439748,0.350668,0.240262,0.091209,0.078276,77.51,0.0,19.329167,38.693889,1.0,69585.0,1.0,4.363226,null
"""max""",172792.0,2.45493,22.057729,9.382558,16.875344,34.801666,73.301626,120.589494,20.007208,15.594995,23.745136,12.018913,7.848392,7.126883,10.526766,8.877742,17.315112,9.253526,5.041069,5.591971,39.420904,27.202839,10.50309,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.16,1.0,23.999444,47.997778,32.0,86398.0,1.0,10.153941,"""lt10"""


## Gridsearch for optimal parameter tunning

In [5]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import numpy as np

In [6]:
# Define the parameter grid to search
param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__max_depth': [3, 5, 7],
    'clf__subsample': [0.7, 0.8, 0.9],
    'clf__colsample_bytree': [0.7, 0.8, 0.9],
    'clf__gamma': [0, 0.1, 0.2]
}

In [7]:
X_train, X_test, y_train, y_test = dataset_pipeline.split(
    test_size=0.3, use_cache=True
)

neg, pos = (len(y_train) - sum(y_train)), sum(y_train)
scale_pos_weight = neg / pos


Loading data from cache...


In [8]:
model_for_tuning = Pipeline(
    [
        (
            "clf",
            XGBClassifier(
                objective="binary:logistic",
                eval_metric="logloss",
                scale_pos_weight=scale_pos_weight,
                random_state=settings.random_state,
                n_jobs=1,
                tree_method='hist',
                device='cuda'
            ),
        )
    ]
)

In [9]:
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=settings.random_state)

grid_search = GridSearchCV(
    estimator=model_for_tuning,
    param_grid=param_grid,
    scoring='recall',
    cv=cv_folds,
    verbose=2,
    n_jobs=1
)

In [10]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


/home/roberto/devel/python/credit-fraud/.venv/lib/python3.13/site-packages/xgboost/core.py:729: UserWarning: [12:26:08] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.7; total time=   2.1s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.7; total time=   1.9s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.7; total time=   1.8s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.7; total time=   1.9s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.7; total time=   1.7s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0.01, clf__max_depth=3, clf__n_estimators=100, clf__subsample=0.8; total time=   1.7s
[CV] END clf__colsample_bytree=0.7, clf__gamma=0, clf__learning_rate=0

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device='cuda',
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
                                                      feature_types=None,
                                                      feature_weig...
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=1,
                                                      num_parallel_tree=None, ...))]),
             n_jobs=1,
             param_grid={'clf__colsample_bytree': [0.7, 0.8, 0.9],
                         'clf__gamma': [0, 0.1, 0.2],
                         'clf__learning_rate': [0.01, 0.05, 0.1],
                         'clf__max_depth': [3, 5, 7],
                         'clf__n_estimators': [100, 200, 300],
                         'clf__subsample': [0.7, 0.8, 0.9]},
             scoring='recall', verbose=2)

In [12]:
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Better recall in cross-validation: {grid_search.best_score_}")

Best parameters found: {'clf__colsample_bytree': 0.7, 'clf__gamma': 0, 'clf__learning_rate': 0.05, 'clf__max_depth': 7, 'clf__n_estimators': 300, 'clf__subsample': 0.7}
Better recall in cross-validation: 1.0
